# Grade Inflation: Data Preparation

## Meta

In [1]:
import pandas as pd
from scipy import stats

In [2]:
path_to_data = './data/hesa_1degree_classification_count.csv'

# import the csv
df_all = pd.read_csv(path_to_data)
df_all.head()

,ID,ACADEMIC YEAR,UKPRN,PROVIDER,CLASSIFICATION,COUNT
0,1,2018,10007811,Bishop Grosseteste University,1,100
1,2,2017,10007811,Bishop Grosseteste University,1,105
2,3,2016,10007811,Bishop Grosseteste University,1,85
3,4,2015,10007811,Bishop Grosseteste University,1,80
4,5,2014,10007811,Bishop Grosseteste University,1,60


## Data processing

We need to calculate the following:

- Mean of proportion of Firsts in 2014
- Variation of proportion of Firsts in 2014
- A sample of 2018

### Create DataFrame

The DataFrame will need the following attributes

- UKPRN
- PROVIDER
- COUNT OF NUMBER OF FIRSTS IN 2014
- TOTAL STUDENTS IN 2014
- PROPORTION OF STUDENTS AWARD FIRSTS IN 2014
- COUNT OF NUMBER OF FIRSTS IN 2018
- TOTAL STUDENTS IN 2018
- PROPORTION OF STUDENTS AWARD FIRSTS IN 2018

In [3]:
# Create a unique list of providers
df_population = df_all[["UKPRN", "PROVIDER"]].drop_duplicates()
df_population.head()

,UKPRN,PROVIDER
0,10007811,Bishop Grosseteste University
5,10001883,De Montfort University
10,10004113,Loughborough University
15,10004797,The Nottingham Trent University
20,10007796,The University of Leicester


In [4]:
# Create temporary views
tmp_2014 = df_all.query('`ACADEMIC YEAR` == 2014')
tmp_2018 = df_all.query('`ACADEMIC YEAR` == 2018')
tmp_first_2014 = tmp_2014.query('CLASSIFICATION == "1"')
tmp_first_2018 = tmp_2018.query('CLASSIFICATION == "1"')

In [5]:
# aggregate total students by provider in tmp_2014 and tmp_2018
tmp_2014 = tmp_2014.groupby(by=["UKPRN"])["COUNT"].sum()
tmp_2018 = tmp_2018.groupby(by=["UKPRN"])["COUNT"].sum()

In [6]:
# merge v_2014 and v_2018
tmp_joined = pd.merge(left=tmp_2014,
                      right=tmp_2018,
                      how="inner",
                      left_on="UKPRN",
                      right_on="UKPRN",
                      suffixes=('_2014', '_2018'))

In [7]:
# preview tmp_joined
tmp_joined.head()

,COUNT_2014,COUNT_2018
UKPRN,,
10000291,3315,4220
10000385,830,1005
10000571,1385,1815
10000712,965,1125
10000824,2885,3585


In [8]:
# join tmp_joined and df_population
df_population = df_population.join(other=tmp_joined,
                                   on="UKPRN",
                                   how="inner")

In [9]:
# preview df_population
df_population.head()

,UKPRN,PROVIDER,COUNT_2014,COUNT_2018
0,10007811,Bishop Grosseteste University,500,455
5,10001883,De Montfort University,4005,6070
10,10004113,Loughborough University,2875,3300
15,10004797,The Nottingham Trent University,5185,6305
20,10007796,The University of Leicester,2880,3415


In [10]:
# Prepare tmp_first_*
tmp_first_2014.rename(columns={"COUNT": "FIRSTS"}, inplace=True)
tmp_first_2014.drop(columns=["ID",
                             "ACADEMIC YEAR",
                             "PROVIDER",
                             "CLASSIFICATION"],
                             inplace=True)

tmp_first_2018.rename(columns={"COUNT": "FIRSTS"}, inplace=True)
tmp_first_2018.drop(columns=["ID",
                             "ACADEMIC YEAR",
                             "PROVIDER",
                             "CLASSIFICATION"],
                             inplace=True)

In [11]:
# merge tmp_first_2014 and tmp_first_2018
tmp_first_joined = pd.merge(left=tmp_first_2014,
                            right=tmp_first_2018,
                            how="inner",
                            left_on="UKPRN",
                            right_on="UKPRN",
                            suffixes=('_2014', '_2018'))

In [12]:
# preview tmp_first_joined
tmp_first_joined.head()

,UKPRN,FIRSTS_2014,FIRSTS_2018
0,10007811,60,100
1,10001883,865,1835
2,10004113,740,960
3,10004797,1110,1100
4,10007796,535,815


In [13]:
# join tmp_first_joined and df_population
df_population = df_population.join(other=tmp_first_joined.set_index("UKPRN"),
                                   on="UKPRN",
                                   how="inner")

In [14]:
# drop objects where 0 exists
df_population = df_population.query('COUNT_2014 != 0')

In [15]:
# Add proportion columns for 2014 and 2018
df_population["PROP 2014"] = df_population["FIRSTS_2014"] / df_population["COUNT_2014"]
df_population["PROP 2018"] = df_population["FIRSTS_2018"] / df_population["COUNT_2018"]

In [16]:
# preview the finished working dataframe
df_population.head()

,UKPRN,PROVIDER,COUNT_2014,COUNT_2018,FIRSTS_2014,FIRSTS_2018,PROP 2014,PROP 2018
0,10007811,Bishop Grosseteste University,500,455,60,100,0.120000,0.219780
5,10001883,De Montfort University,4005,6070,865,1835,0.215980,0.302306
10,10004113,Loughborough University,2875,3300,740,960,0.257391,0.290909
15,10004797,The Nottingham Trent University,5185,6305,1110,1100,0.214079,0.174465
20,10007796,The University of Leicester,2880,3415,535,815,0.185764,0.238653


In [17]:
# store the variable
%store df_population

Stored 'df_population' (DataFrame)


## Sample the DataFrame

In [18]:
# take a 33% random sample of the dataframe
df_working["SAMPLE"] = stats.bernoulli.rvs(p=0.33, size=df_working["UKPRN"].size)
# select only successful picks
df_sampled = df_working.query('SAMPLE == 1')

NameError: name 'df_working' is not defined

In [ ]:
df_sampled.head()

## Visualisations

In [ ]:
df_sampled[["PROP 2014"]].plot(kind="hist")

In [ ]:
df_sampled[["PROP 2018"]].plot(kind="hist")

In [ ]:
df_sampled[["PROP 2014", "PROP 2018"]].plot(kind="box")

In [ ]:
df_sampled.plot(kind="scatter",
                x="PROP 2014",
                y="PROP 2018")

## Analysing the confidence interval

In [ ]:
# sample mean 2014
mean_2014 = df_sampled["PROP 2014"].mean()
# sample mean 2018
mean_2018 = df_sampled["PROP 2018"].mean()
# estimated standard error 2014.
est_std_err = df_sampled["PROP 2014"].std()/(df_sampled["UKPRN"].size) ** 0.5

In [ ]:
# declare the normal
z = stats.norm(loc=mean_2014, scale=est_std_err)

In [ ]:
# calculate the Pr(mean_2018)
Pr = round(2 * (1 - z.cdf(x=mean_2018)), 6)

In [ ]:
# get the 95% CI for sample mean 2014
ci = z.interval(alpha=0.95)

In [ ]:
# Test if mean 2018 in the confidence interval
outcome = mean_2018 > ci[0] and mean_2018 < ci[1]

In [ ]:
print("Summary\n-------")
print("Outcome:", outcome)
print("ci: (", round(ci[0], 3), ",", round(ci[1], 3),")")
print("mean 2014:", round(mean_2014, 3))
print("mean 2018:", round(mean_2018, 3))
print("Pr():", Pr)